In [ ]:
import pandas as pd
import numpy as np
from data_quality_tests import DataQuality as dq

<h3>Limpeza e ajustes dos dados<h3>

In [ ]:
# Carregar os dados
df = pd.read_csv('Telco-Customer-Churn.csv')

In [ ]:
# Verificar colunas e as primeiras linhas dos dados
df.head().T

O data set inclui informações sobre:

- Clientes que saíram no último mês: coluna Churn
- Serviços que cada cliente se inscreveu: telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes
- Informações da conta do cliente: há quanto tempo ele é cliente, contrato, método de pagamento, cobrança sem papel, cobranças mensais e cobranças totais
- Informações demográficas sobre os clientes – sexo, faixa etária e se têm parceiros e dependentes

In [ ]:
# Verificação das dimensões do dataset
df.shape

In [ ]:
# Checando qualidade com a biblioteca DataQuality
dq.data_quality_check(df)

In [ ]:
# Verificar informações sobre as colunas e tipos de dados
df.info()

In [ ]:
#Converter o tipo de dado de TotalCharges para Numérico 
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [ ]:
#Verificar valores nulos
df.isnull().sum()

In [ ]:
#Remover nulos
df.dropna(inplace = True)

In [ ]:
dq.data_quality_check(df)

In [ ]:
#Identificar Outliers
dq.outlier_columns(df)

In [ ]:
import plotly.express as px
fig = px.box(df, y="tenure")
fig.show()

In [ ]:
fig = px.box(df, y="MonthlyCharges")
fig.show()

In [ ]:
fig = px.box(df, y="TotalCharges")
fig.show()

Nos graficós acima podemos ver que os outliers não são significativos, pois não aparecem explicitamente no gráfico. 

In [ ]:
df.describe()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles=("Monthly Charges", "Total Charges", "Tenure"))

trace1 = go.Histogram(x=df['MonthlyCharges'])
trace2 = go.Histogram(x=df['TotalCharges'])
trace3 = go.Histogram(x=df['tenure'])



fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.add_trace(trace3, row=1, col=3)

fig.update_layout(height=400, width=1000)

fig.show()

Temos que levar em consideração que nenhuma das 3 colunas é normalmente distribuída
<br>Monthly Charges: muitas contas mensais com valores baixos. Automaticamente o Total Charges terá também contas com valores totais baixos.
<br>Tenure: quantidade de meses que o cliente está na empresa. Muitos clientes novos, mas também uma quantidade de clientes que pernaneceram na empresa por mais de 70 meses.
<br>Em resumo, não devemos nos preocupar com as anomalias apontadas pela biblioteca dataQuality

<h3>Análise Exploratória dos Dados<h3>

Verificando os dados demográficos dos clientes

In [ ]:
df['gender'].value_counts()*100/len(df)

A distribuição do gêneros dos clientes é bem dividida

In [ ]:
df['SeniorCitizen'].value_counts()*100/len(df)

A maior parte dos clientes são jovens. Apenas 16% dos clientes são idosos.

In [ ]:
df['Partner'].value_counts()*100/len(df)

48% dos clientes tem parceiro

In [ ]:
df['Dependents'].value_counts()*100/len(df)

enquanto 30% dos clientes tem dependentes

In [ ]:
partner_dependents = df.groupby(['Partner','Dependents']).size().unstack()
partner_dependents_percent = (partner_dependents.T*100.0 / partner_dependents.T.sum()).T

partner_dependents_percent_str = partner_dependents_percent.applymap('{:.2f}%'.format)

partner_dependents_percent_str.style

Entre os clientes que possuim parceiro, apenas metade deles também possui dependente. E, como esperado, dos cliente sem parceiro 90% não possui dependentes.

In [ ]:
df['Contract'].value_counts()*100/len(df)

A maioria dos clientes estão com contratos mensais. 

In [ ]:
df.columns

In [ ]:
#Verificar distribuição de Serviços 
dfs = []
services = ['PhoneService','MultipleLines','InternetService','OnlineSecurity',
           'OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

for item in services:

    value_counts = df[item].value_counts()*100/len(df)

    df2 = pd.DataFrame({'value': value_counts.index, 'count': value_counts.values})
    df2.rename(columns={'count': item}, inplace=True)
    dfs.append(df2)


table = pd.concat(dfs, axis=1)

table = table.sort_index()

print(table)

<h4>Distribuição da variável target Churn</h4>

In [ ]:
df.Churn.value_counts()

Comparar variáveis categórias com variável target Churn

In [ ]:
#Função para comparar váriáveis categóricas com variável target Churn
def categorical_vs_churn(column):
    table = df.groupby([column])['Churn'].value_counts().unstack().divide(df.groupby([column])['Churn'].value_counts().unstack().sum(axis=1),axis=0)*100
    table = table.applymap("{:.2f}%".format)
    return table

Gender vs Churn

In [ ]:
categorical_vs_churn('gender')

Senior Citzen vs Churn

In [ ]:
categorical_vs_churn('SeniorCitizen')

Partner vs Churn

In [ ]:
categorical_vs_churn('Partner')

Dependents vs Churn

In [ ]:
categorical_vs_churn('Dependents')

Phone service vs Churn

In [ ]:
categorical_vs_churn('PhoneService')

Multiple lines vs Churn

In [ ]:
categorical_vs_churn('MultipleLines')

Internet Service vs Churn

In [ ]:
categorical_vs_churn('InternetService')

Online Security vs Churn

In [ ]:
categorical_vs_churn('OnlineSecurity')

Online Backup vc Churn

In [ ]:
categorical_vs_churn('OnlineBackup')

Device protection vs Churn

In [ ]:
categorical_vs_churn('DeviceProtection')

Tech Support vs Churn


In [ ]:
categorical_vs_churn('TechSupport')

Streaming TV vs Churn

In [ ]:
categorical_vs_churn('StreamingTV')

Streaming Movies vs churn

In [ ]:
categorical_vs_churn('StreamingMovies')

Contract vs Churn

In [ ]:
categorical_vs_churn('Contract')

Paperless billing vs churn

In [ ]:
categorical_vs_churn('PaperlessBilling')

Payment method vs churn

In [ ]:
categorical_vs_churn('PaymentMethod')

Comparar váriáveis numéricas com variável target Churn

In [ ]:
df['MonthlyChargesBucket'] = pd.cut(df['MonthlyCharges'], bins=range(0, 121, 20), right=False)

freq_table = pd.crosstab(df['MonthlyChargesBucket'], df['Churn'])
freq_table

In [ ]:
df['TotalChargesBucket'] = pd.cut(df['TotalCharges'], bins=range(0, 9000, 1000), right=False)

# Criando a tabela de frequência
freq_table = pd.crosstab(df['TotalChargesBucket'], df['Churn'])
freq_table

In [ ]:
df['TenureBucket'] = pd.cut(df['tenure'], bins=range(0, 80, 5), right=False)

# Criando a tabela de frequência
freq_table = pd.crosstab(df['TenureBucket'], df['Churn'])
freq_table

<h4>Probabilidade de churn</h4>
Gender: sem efeitos<br>
SeniorCitizen: idosos são mais propensos a churn<br>
Partner/Dependent: clientes com parceiros/dependentes são menos propensos a churn<br>
PhoneService: sem efeitos<br>
MultipleLines: clientes com múltiplas linhas são mais propensos a churn<br>
InternetService: clientes com fibra óptica são mais propensos a churn e clientes com DSL são mais propensos a churn do que aqueles sem serviço de internet<br>
OnlineSecurity/OnlineBackup/DeviceProtection/TechSupport: clientes sem esses serviços são mais propensas a churn<br>
StreamingTV/StreamingMovies: sem efeito<br>
Contract: clientes com contratos mensais são mais propensas a churn. Clientes com contrato de 2 anos são menos propensas a churn<br>
PaperlessBilling: clientes que adotam a fatura digital estão mais propensos a churn<br>
PaymentMethod: clientes que efetuam pagamentos com electronic check são mais propensos a churn<br>

Tenure: quanto mais tempo o cliente permanece no provedor, menor a  propensão a churn<br>
MonthlyCharges: à medida que as cobranças mensais aumentam, a probabilidade de churn aumenta gradativamente<br>
TotalCharges: conforme as cobranças totais aumentam, a probabilidade de churn diminui. Provavelmente por conta do "Tenure"

<h3>Preparar Dados para ML</h3>

In [ ]:
df.columns

In [ ]:
#deletar colunas que não vamos usar
df = df.drop(columns=['customerID','gender','PhoneService','MultipleLines','TotalCharges','SeniorCitizen','Partner','Dependents','PaperlessBilling','MonthlyChargesBucket', 'TotalChargesBucket', 'TenureBucket'], axis=1)

Alterar variáveis categóricas em númericas

In [ ]:
df['Churn'] = df['Churn'].replace({'No':0,'Yes':1})

In [ ]:
y = df['Churn']
x = df.drop('Churn', axis = 1)

In [ ]:
x = pd.get_dummies(x)

In [ ]:
x.head()

In [ ]:
df.Churn.value_counts()

Variável target desbalanceada. Há mais clientes que não cancelaram os serviços do que clientes que cancelaram.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [ ]:
# Dividir conjunto de dados em treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
# verificar tamanho dos datasets de treino e teste
print("X train", x_train.shape)
print("Y train", y_train.shape)
print("X test", x_test.shape)
print("Y test", y_test.shape)


In [ ]:
y_train.value_counts()

In [ ]:
from tabulate import tabulate

# Modelos a serem avaliados
models = {'Logistic Regression': LogisticRegression(max_iter=1000),
          'Decision Tree': DecisionTreeClassifier(),
          'Random Forest': RandomForestClassifier(),
          'Gradient Boosting': GradientBoostingClassifier(),
           'Ada Boosting': AdaBoostClassifier()}


results = []

for model_name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    results.append([model_name,
                    accuracy_score(y_test, y_pred),
                    precision_score(y_test, y_pred),
                    recall_score(y_test, y_pred),
                    f1_score(y_test, y_pred),
                    confusion_matrix(y_test, y_pred),
                    roc_auc_score(y_test, y_pred)])

# Tabela com os resultados
headers = ['Model', 'Accuracy Score', 'Precision Score', 'Recall Score', 'F1 Score', 'Confusion Matrix', 'AUC-ROC']
table = tabulate(results, headers=headers)

print(table)

O modelo que obteve o melhor resultado foi o de <u>Logistic Regression</u>

Aplicar modelo de Logistic Regression nos dados

In [ ]:
model = LogisticRegression()
# Treino
model.fit(x_train, y_train)

# Previsões no conjunto de teste
y_pred = model.predict(x_test)


Aplicando novos dados e exibindo as predições

In [ ]:
new_data = pd.read_csv('novos_dados.csv', sep=';')

new_data = new_data.drop(columns=['gender','PhoneService','MultipleLines','TotalCharges','SeniorCitizen','Partner','Dependents','PaperlessBilling'], axis=1)
new_data.dropna(inplace = True)

# Codificar as variáveis categóricas, excluindo o id do cliente
data_encoded = pd.get_dummies(new_data.iloc[:, 1:])

# Aplicando o modelo para extrair as predições
predictions = pd.DataFrame(model.predict(data_encoded),columns=['Churn Prediction'])

# Concatenar o ID do cliente com as previsões
combined_df = pd.concat([new_data, predictions], axis=1)
combined_df['Churn Prediction'] = combined_df['Churn Prediction']
combined_df['Churn Prediction'] = combined_df['Churn Prediction'].replace({0:'No', 1:'Yes'})


print(combined_df)
combined_df.to_csv('predicoes.csv', index=False)
